In [ ]:
# pip install google-generativeai
# pip install rouge

import google.generativeai as genai
import json
import pandas as pd
import numpy as np
import os

from dotenv import load_dotenv
from rouge import Rouge

In [38]:
load_dotenv()
genai.configure(api_key=os.environ['GEMINI_KEY'])
DIRECTORY_PATH = os.environ['DATA_PATH']
GEMINI_VERSION = "gemini-1.5-flash"

cleaned_fn = "cleaned.json"
ocred_fn = "original_ocr.json"


datasetRaw = {}

for file_name in [ocred_fn, cleaned_fn]:
    if file_name not in os.listdir(DIRECTORY_PATH):
        print(f"ERROR 404 ! File {file_name} not Found...")

    file_path = os.path.join(DIRECTORY_PATH, file_name)
    with open(file_path, 'r') as file:
        datasetRaw[file_name.split('.')[0]] = json.load(file)
        file.close()


### Output evaluation

In [3]:
OUTPUT_PATH = os.environ['OUTPUT_PATH']

modes = [
    {
        "name":"lama",
        "folder":"Llama"
    },
    {
        "name":"minerva",
        "folder":"Minerva"
    }
]

In [76]:
def segment_wrapper(segment_ocred, segment_clean):
    return f"""
        ### Task: Il primo testo è una versione del secondo estratto con oc-Red, ed è stata processata per ridurre gli errori, valuta con un punteggio tra 1 e 100 tenendo conto di correttezza, comprensibilità e somiglianza
        ### Testo da valutare: {segment_ocred}.
        ### Testo di confronto: {segment_clean}.
        ### Requisiti:
            - Scrivi il risultato in formato <criterio>:<punteggio>.
            - Non usare altri numeri interi o fai altri commenti
        ### Risultato:
    """

In [77]:
results = {}
baseline = {}
gemini_eval = {}
for mode in modes:
    model_scorer = Rouge()
    model_outputs = os.path.join(OUTPUT_PATH, mode['folder'])
    gemini = genai.GenerativeModel(model_name=GEMINI_VERSION)
    corrected_chapters = []
    baseline_chapters = []
    reference_chapters = []
    gemini_eval[mode['name']] = []
    for n, chapter in enumerate(os.listdir(model_outputs)):
        complete_chapter = ""
        with open(os.path.join(model_outputs, chapter), mode='r', encoding='utf-8') as file:
            try:
                chapter_object = json.load(file)
            except Exception as e:
                print(e, f"while trying to read {mode['folder']}-{chapter}")
                continue
            for line in chapter_object:
                complete_chapter += ("\n"+line['output'])
            file.close()
        corrected_chapters.append(complete_chapter)
        reference_chapters.append(datasetRaw['cleaned'][f'{n+1}'])
        baseline_chapters.append(datasetRaw['original_ocr'][f'{n+1}'])
        evaluation = gemini.generate_content(segment_wrapper(complete_chapter, datasetRaw['cleaned'][f'{n+1}']))
        gemini_eval[mode['name']].append(evaluation.text)
    results[mode['name']] = model_scorer.get_scores(corrected_chapters,reference_chapters, avg=True)
    baseline[mode['name']] = model_scorer.get_scores(baseline_chapters, reference_chapters, avg=True)
    
print(f"{'-'*20}")
print(results)
print(baseline)
print(f"{'-'*20}")


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 8
}
]

In [ ]:
llama_res = pd.DataFrame(results['lama'])
llama_base = pd.DataFrame(baseline['lama'])
minerva_res = pd.DataFrame(results['minerva'])
minerva_base = pd.DataFrame(baseline['minerva'])


In [ ]:
print(llama_res)
print(llama_base)
print(minerva_res)
print(minerva_base)

   rouge-1   rouge-2  rouge-l
r  0.90181  0.861196  0.90181
p  0.88430  0.846039  0.88430
f  0.89291  0.853540  0.89291
    rouge-1   rouge-2   rouge-l
r  0.866203  0.809943  0.866203
p  0.799548  0.765606  0.799548
f  0.831489  0.787136  0.831489
    rouge-1   rouge-2   rouge-l
r  0.888744  0.844015  0.888744
p  0.880486  0.845781  0.880486
f  0.884081  0.844281  0.884081
    rouge-1   rouge-2   rouge-l
r  0.866203  0.809943  0.866203
p  0.799548  0.765606  0.799548
f  0.831489  0.787136  0.831489


In [ ]:
for i, content in enumerate(gemini_eval['lama']):
    print(i+1, content)

1 Correttezza:75
Comprensibilità:90
Somiglianza:95

2 Correttezza: [90]
Comprensibilità: [95]
Somiglianza: [98]

3 Correttezza: [90]
Comprensibilità: [95]
Somiglianza: [98]

4 Correttezza:85
Comprensibilità:90
Somiglianza:95

5 Correttezza: [90]
Comprensibilità: [95]
Somiglianza: [98]

6 Correttezza:90
Comprensibilità:95
Somiglianza:98

7 Correttezza:75
Comprensibilità:90
Somiglianza:95



In [ ]:
for i, content in enumerate(gemini_eval['minerva']):
    print(i+1, content)

1 Correttezza:85
Comprensibilità:90
Somiglianza:95

2 Correttezza: [90]
Comprensibilità: [95]
Somiglianza: [98]

3 Correttezza: [90]
Comprensibilità: [95]
Somiglianza: [98]

4 Correttezza:85
Comprensibilità:90
Somiglianza:95

5 Correttezza: [90]
Comprensibilità: [95]
Somiglianza: [98]

6 Correttezza:85
Comprensibilità:90
Somiglianza:95

7 Correttezza:85
Comprensibilità:90
Somiglianza:95

